## Data Analysis Using SQL
***

<font color=blue>The analysis below was done for a project when I was learning to use SQL. The SQL code analyzes lipid panel results for determining whether patients may be at risk of coronary artery disease (CAD). The data and analysis are based on simplified real life data, and does not reflect all factors and steps used in the process of evaluating patients for CAD.

The database (AHD.db) contains three tables: patients, panels, recommended_values. The tables will be used to answer various questions, culminating in determing which patients may be at risk based on their LDL scores.

**Remember to use the file "AHD_copy_original.db". Save it to your local computer as "AHD.db". If you save using another file name, just update the file name in the sqlite path below.**
</font>
***

### Load sqlite for use in Jupyter Notebook
***

<font color=green>If you wish to run the code: the cell below will enable local use of sqlite. The path (sqlite:////...) should be replaced with the path/location of where you saved the db file on your local machine. After loading and establishing a link to the database, sql magic symbols mustbe used in every code cell: "%sql" (for one line of code) or "%%sql" (for multiple lines of code).

Note that importing sqlite3 and pandas could be another approach to using sql in jupyter notebook.</font>

In [1]:
%load_ext sql
%sql sqlite:////Users/kvd51/Documents/Git/SQL/AHD/AHD.db

### Review each table

<font color=green>First let's take a look at each of the three tables: patients, panels, recommended_values. Output from the patients and panels tables is limited to 7 rows using "LIMIT".</font>

In [9]:
%%sql
SELECT *
FROM patients
LIMIT 7;

 * sqlite:////Users/kvd51/Documents/Git/SQL/AHD.db
Done.


ID,Name,Address,Phone,Email,Status
1,Karlis Clemes,2507 Tony Point,801-246-9110,kclemes0@businesswire.com,inactive
2,Garnet Wadesworth,436 Ryan Crossing,763-828-5228,gwadesworth1@e-recht24.de,active
3,Carroll Dickins,04422 Dovetail Alley,467-846-7771,cdickins2@wordpress.com,inactive
4,Carlee Rylett,46506 Summerview Center,917-247-3933,crylett3@reddit.com,inactive
5,Vinita Ruller,5095 Dovetail Terrace,416-861-9250,vruller4@163.com,inactive
6,Yvor Brownjohn,9 Logan Terrace,602-245-3866,ybrownjohn5@hexun.com,active
7,Valaria Broadwell,4808 Hollow Ridge Avenue,740-299-5939,vbroadwell6@dropbox.com,inactive


In [10]:
%%sql
SELECT *
FROM panels
LIMIT 7;

 * sqlite:////Users/kvd51/Documents/Git/SQL/AHD.db
Done.


ID,Cholesterol,HDL,Triglycerides,patient_ID
1,230.61,71.87,81.82,97
2,294.04,93.98,123.89,48
3,250.1,78.31,237.73,55
4,122.01,55.05,166.37,30
5,281.93,81.55,223.15,14
6,52.73,26.33,115.23,64
7,224.44,86.57,200.34,41


In [11]:
%%sql
SELECT *
FROM recommended_values;

 * sqlite:////Users/kvd51/Documents/Git/SQL/AHD.db
Done.


ID,Lipid,Low,Desirable,Borderline_High,High
1,Total_Cholesterol,NULL,NULL,200,240
2,LDL,NULL,NULL,130,160
3,HDL,NULL,50,NULL,NULL
4,Triglycerides,NULL,NULL,150,200


### Make some alterations

<font color=green>For the patient with ID = 7, we want to change the status from inactive to active, then view the row of data for patient 7 to confirm that the status was changed.
    </font>

In [2]:
%%sql
UPDATE patients
SET Status = 'active'
WHERE ID = 7;

SELECT *
FROM patients
WHERE ID = 7;

 * sqlite:////Users/kvd51/Documents/Git/SQL/AHD.db
1 rows affected.
Done.


ID,Name,Address,Phone,Email,Status
7,Valaria Broadwell,4808 Hollow Ridge Avenue,740-299-5939,vbroadwell6@dropbox.com,active


<font color=green>Now let's split the patients table into two tables: active_patients and inactive_patients
    The inactive_patients table is created first by selecting the inactive rows from the patients table.
    </font>

In [3]:
%%sql
CREATE TABLE inactive_patients
AS  SELECT *
    FROM patients
    WHERE Status = 'inactive';

 * sqlite:////Users/kvd51/Documents/Git/SQL/AHD.db
Done.


[]

<font color=green>Count the number of IDs in the inactive_patients table.
    </font>

In [6]:
%%sql
SELECT COUNT(ID) AS 'num_inactive'
FROM inactive_patients;

 * sqlite:////Users/kvd51/Documents/Git/SQL/AHD.db
Done.


num_inactive
52


<font color=green>Delete the inactive patients from the patients table, and rename the table to "active_patients" since only active patients are left in the table.
    </font>

In [4]:
%%sql
DELETE FROM patients
WHERE Status = 'inactive';

ALTER TABLE patients
RENAME TO active_patients;

 * sqlite:////Users/kvd51/Documents/Git/SQL/AHD.db
52 rows affected.
Done.


[]

<font color=green>Verify that the active_patients table contains only active patients. All columns are selecting from the table; rows are limited to 5.
    </font>

In [2]:
%%sql
SELECT *
FROM active_patients
LIMIT 5;

 * sqlite:////Users/kvd51/Documents/Git/SQL/AHD.db
Done.


ID,Name,Address,Phone,Email,Status
2,Garnet Wadesworth,436 Ryan Crossing,763-828-5228,gwadesworth1@e-recht24.de,active
6,Yvor Brownjohn,9 Logan Terrace,602-245-3866,ybrownjohn5@hexun.com,active
7,Valaria Broadwell,4808 Hollow Ridge Avenue,740-299-5939,vbroadwell6@dropbox.com,active
10,Rossy McGebenay,40 Elka Pass,474-164-2534,rmcgebenay9@goodreads.com,active
11,Gordie Longdon,8 Bartelt Pass,616-774-3837,glongdona@i2i.jp,active


### Data analysis done to fulfill random requests

<font color=green>Someone wants to know the percentage of active and inactive patients. This query uses WITH to count the active, inactive and total patients and asigns the counts to aliases. The aliases are then referenced to calculate the percentages.
    </font>

In [13]:
%%sql
WITH
total_active AS (
    SELECT COUNT(ID) AS active
    FROM active_patients
),
total_inactive AS (
    SELECT COUNT(ID) AS inactive
    FROM inactive_patients
),
total_patients AS (
    SELECT total_active.active + total_inactive.inactive AS total
    FROM total_active, total_inactive
)
SELECT (total_active.active * 100 / total_patients.total) AS active_percent,
        (total_inactive.inactive * 100 / total_patients.total) AS inactive_percent
FROM total_active, total_inactive, total_patients;


 * sqlite:////Users/kvd51/Documents/Git/SQL/AHD.db
Done.


active_percent,inactive_percent
48,52


<font color=green>Someone wants to know the last active patient seen by using the MAX aggregate.
    </font>

In [16]:
%%sql
SELECT *
FROM active_patients
WHERE ID = (
    SELECT MAX(ID) FROM active_patients);

 * sqlite:////Users/kvd51/Documents/Git/SQL/AHD.db
Done.


ID,Name,Address,Phone,Email,Status
99,Daisy Foden,04 Pine View Terrace,407-423-4561,dfoden2q@cbslocal.com,active


<font color=green>Someone wants to randomly select ten patients (active or inactive). Given that the patients table was split into active and inactive, we'll temporarily combine them with UNION. The random() function is used in combination with ORDER BY to randomly order the list of patients, then the top ten patients are given by using LIMIT.
    </font>

In [23]:
%%sql
SELECT * 
FROM (
    SELECT *
    FROM active_patients
    UNION
    SELECT *
    FROM inactive_patients
)
ORDER BY RANDOM()
LIMIT 10;

 * sqlite:////Users/kvd51/Documents/Git/SQL/AHD.db
Done.


ID,Name,Address,Phone,Email,Status
97,Chas Cicci,109 Drewry Hill,904-701-1413,ccicci2o@vistaprint.com,active
80,Flin Lodovichi,7 Anderson Way,301-591-3154,flodovichi27@blinklist.com,active
28,Morris Whetland,72 Dexter Circle,133-259-1189,mwhetlandr@rambler.ru,active
33,Nealy Camblin,89671 Killdeer Terrace,436-337-4886,ncamblinw@icio.us,active
48,Jazmin Sholl,743 Hudson Way,286-963-5049,jsholl1b@walmart.com,inactive
20,Myron Atkin,7523 Lillian Park,301-819-0545,matkinj@irs.gov,inactive
98,Arlinda Hoult,4 Prairie Rose Center,949-378-2061,ahoult2p@globo.com,inactive
87,Krista Kaes,8 Judy Street,743-910-7495,kkaes2e@globo.com,inactive
42,Freddie Garstang,0778 Marcy Way,491-180-9446,fgarstang15@oakley.com,active
69,Elsbeth Workman,3 Lawn Street,991-910-4412,eworkman1w@mozilla.org,inactive


<font color=green>Someone wants to know the patient IDs of patients that have had more than one panel done. The query below selects patient_ID and a count of patient_ID, then uses HAVING to set the count condition (count > 1). The resulting table is ordered by the count of patient_ID, in descending order.
    </font>

In [31]:
%%sql
SELECT patient_ID, COUNT(patient_ID)
FROM panels
GROUP BY patient_ID
HAVING COUNT(patient_ID) > 1
ORDER BY 2 DESC;

 * sqlite:////Users/kvd51/Documents/Git/SQL/AHD.db
Done.


patient_ID,COUNT(patient_ID)
14,5
11,4
48,4
16,3
28,3
32,3
46,3
56,3
65,3
2,2


### Data analysis of CAD risk

<font color=green>LDL is the "bad" cholesterol, and will be used to determine CAD risk. However, LDL is not included in the panels table, so it will have to be calculated.
    
First we'll add an LDL column to the panels table.
    </font>

In [36]:
%%sql
ALTER TABLE panels ADD COLUMN LDL 
DECIMAL(5, 2);

 * sqlite:////Users/kvd51/Documents/Git/SQL/AHD.db
Done.


[]

<font color=green>Next, we'll add the calculated LDL to the panels table. The calculation used is called the Friedewald equation, which was used historically used to calculate LDL.
    </font>

In [41]:
%%sql
UPDATE panels
SET LDL = ROUND(panels.Cholesterol - (panels.HDL + panels.Triglycerides/5), 2);

 * sqlite:////Users/kvd51/Documents/Git/SQL/AHD.db
100 rows affected.


[]

<font color=green>Next, display the panels table to confirm the LDL levels were added and appear correct.
    </font>

In [42]:
%%sql
SELECT *
FROM panels
LIMIT 5;

 * sqlite:////Users/kvd51/Documents/Git/SQL/AHD.db
Done.


ID,Cholesterol,HDL,Triglycerides,patient_ID,LDL
1,230.61,71.87,81.82,97,142.38
2,294.04,93.98,123.89,48,175.28
3,250.1,78.31,237.73,55,124.24
4,122.01,55.05,166.37,30,33.69
5,281.93,81.55,223.15,14,155.75


<font color=green>Now we'll provide a list of patients that may be at risk of CAD:
 - We'll use WITH to select and alias out the Borderline_High LDL value (the threshold value to measure against the calculated LDLs) from the recommended_values table. The alias for the selection is 'rv.' 
 - Someone requested the Name, LDL, Address and Phone for the at risk patients, so the query will need to JOIN the active_patients and panels tables to provide all the requested fields. The JOIN will be ON patient_ID. 
 - HAVING is used to compare the LDL levels against the Borderline_High value from the rv alias (rv.Borderline_High).
 - Selected patient information is returned if the LDL level is >= the LDL Bordeline_High value.
    </font>

In [52]:
%%sql
WITH rv AS (
    SELECT Borderline_High
    FROM recommended_values
    WHERE Lipid = 'LDL'
)
SELECT Name, LDL, Address, Phone
FROM active_patients, rv
JOIN panels
    ON active_patients.ID = panels.patient_ID
GROUP BY LDL, Name
HAVING LDL >= rv.Borderline_High
ORDER BY LDL DESC;

 * sqlite:////Users/kvd51/Documents/Git/SQL/AHD.db
Done.


Name,LDL,Address,Phone
Vikky Rudkin,199.41,9 Heath Point,242-283-8676
Talbert Froom,195.24,8 Ludington Alley,977-720-5869
Freddie Garstang,183.61,0778 Marcy Way,491-180-9446
Moreen Corse,175.71,4 Lake View Alley,606-453-2457
Morris Whetland,170.82,72 Dexter Circle,133-259-1189
Melisandra Southward,161.33,6776 Harper Avenue,427-110-3840
Shalne Paddemore,157.43,17 Green Lane,955-675-7880
Anastasia Bartolomieu,155.75,57 Fallview Street,326-441-6684
Hunt Collumbine,146.61,15 Toban Drive,730-309-1736
Chas Cicci,142.38,109 Drewry Hill,904-701-1413


<font color=green>Finally, we want to know the number of patients that have LDL levels above the High threshold and the number above the Borderline_High but less than High. This is more complicated. We'll need to use CASE (like an IF statement).
 - Need to first use WITH to select and alias out Borderline_High and High LDL values from the recommended_values table.
 - Select the count of LDL levels greater than the recommended values from the panels table. Alias the counts to make it clear what the count values are when they are returned in the output.
    </font>

In [55]:
%%sql
WITH rv AS (
    SELECT Borderline_High, High
    FROM recommended_values
    WHERE Lipid = 'LDL'
)
SELECT
    COUNT(
        CASE
            WHEN(
                LDL >= rv.High
            )
            THEN LDL
        END
    ) AS High,
    COUNT(
        CASE
            WHEN(
                LDL >= rv.Borderline_High AND LDL < rv.High
            )
            THEN LDL
        END
    ) AS Borderline_High
FROM panels, rv;

 * sqlite:////Users/kvd51/Documents/Git/SQL/AHD.db
Done.


High,Borderline_High
12,9
